# Web Scraping data from books website

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_soup(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure request was successful
    return BeautifulSoup(response.text, 'html.parser')

# Extract category from book detail page
def get_book_category(book_url):
    soup = get_soup(book_url)
    # Breadcrumb: Home > Books > Category > Book Title
    breadcrumb = soup.find('ul', class_='breadcrumb')
    if breadcrumb:
        links = breadcrumb.find_all('a')
        # 0: Home, 1: Books, 2: Category
        if len(links) >= 3:
            return links[2].text.strip()
    return 'Unknown'

def scrape_books():
    base_url = "https://books.toscrape.com/"
    books_data = []
    page = 1
    
    try:
        while True:
            # Construct URL for each page
            if page == 1:
                url = base_url
            else:
                url = base_url + f'catalogue/page-{page}.html'
            
            print(f"Scraping page {page}...")
            response = requests.get(url, headers=headers)
            
            # Break if page not found
            if response.status_code == 404:
                break
                
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all books on the page
            books = soup.select('article.product_pod')
            
            if not books:
                break
                
            for book in books:
                product_url = base_url + "catalogue/" + book.select_one('h3 a')['href'].replace('../', '').replace("catalogue", "")
                book_data = {
                    "title": book.h3.a['title'],
                    "price": book.select_one('p.price_color').text.strip(),
                    "rating": book.select_one('p.star-rating')['class'][1],
                    "availability": book.select_one('p.availability').text.strip(),
                    "image_url": base_url + book.select_one('img')['src'].replace('../', ''),
                    "product_url": product_url,
                    "category": get_book_category(product_url)
                }
                books_data.append(book_data)
            
            # Add delay to be respectful to the server
            # time.sleep(1)
            page += 1
        
        # Save to JSON file
        with open('books_datav2.json', 'w', encoding='utf-8') as f:
            json.dump(books_data, f, indent=4, ensure_ascii=False)
            
        print("\nData successfully scraped and saved to books_data.json")
        return books_data
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Run the scraper
if __name__ == "__main__":
    books_data = scrape_books()
    if books_data:
        print(f"\nTotal books scraped: {len(books_data)}")
        print("\nSample Book Data:")
        print(json.dumps(books_data[0], indent=2))

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

## Converting the JSON file to XML and validation using RelaxNG

In [7]:
# Import required libraries
from lxml import etree
import json
from typing import Dict, List
import os

def create_book_element(doc: etree.ElementTree, book_data: Dict) -> etree.Element:
    """Create an XML element for a book from its JSON data"""
    book = etree.SubElement(doc, "book")
    
    # Add all book details as sub-elements
    for key, value in book_data.items():
        elem = etree.SubElement(book, key.replace("_", "-"))
        elem.text = str(value)
    
    return book

def validate_xml(xml_file: str, schema_file: str) -> bool:
    """
    Validate XML against RelaxNG schema
    Returns True if valid, False otherwise
    """
    try:
        # Verify files exist
        if not os.path.exists(xml_file):
            raise FileNotFoundError(f"XML file not found: {xml_file}")
        if not os.path.exists(schema_file):
            raise FileNotFoundError(f"Schema file not found: {schema_file}")
            
        # Parse the XML file and schema
        xml_doc = etree.parse(xml_file)
        relaxng_doc = etree.RelaxNG(etree.parse(schema_file))
            
        # Validate and return result
        is_valid = relaxng_doc.validate(xml_doc)
        
        if is_valid:
            print("✅ XML document is valid against the schema")
        else:
            print("❌ XML validation failed!")
            for error in relaxng_doc.error_log:
                print(f"Line {error.line}: {error.message}")
            
        return is_valid
        
    except Exception as e:
        print(f"Validation error: {str(e)}")
        return False

def json_to_xml(json_file: str, xml_file: str):
    """Convert JSON book data to XML format and validate"""
    try:
        # Read JSON data
        with open(json_file, 'r', encoding='utf-8') as f:
            books_data = json.load(f)
        
        # Create root element
        root = etree.Element("books")
        
        # Add each book to the XML tree
        for book_data in books_data:
            create_book_element(root, book_data)
        
        # Create XML tree and save to file
        tree = etree.ElementTree(root)
        tree.write(
            xml_file, 
            pretty_print=True, 
            encoding='utf-8', 
            xml_declaration=True
        )
        
        print(f"Successfully converted JSON to XML. Saved as {xml_file}")
        
        # Validate the XML
        if validate_xml(xml_file, 'books_schema.rng'):
            print("XML structure is valid")
            return True
        else:
            print("XML structure is invalid")
            return False
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

# Run the conversion and validation
if __name__ == "__main__":
    result = json_to_xml('books_datav2.json', 'books_datav2.xml')
    print(f"\nConversion and validation {'successful' if result else 'failed'}")

Successfully converted JSON to XML. Saved as books_datav2.xml
✅ XML document is valid against the schema
XML structure is valid

Conversion and validation successful


## Converting the Natural Language to Xquery

In [ ]:
import re
from lxml import etree
import subprocess

xml_file_path = "./books_datav2.xml"  
with open(xml_file_path, 'rb') as f:
    xml_data = f.read()
tree = etree.XML(xml_data)

def nl_to_xquery(nl_query):

    query = nl_query.lower().strip()

    if "all book titles" in query or "list book titles" in query:
        return "for $b in /books/book\nreturn $b/title"

    if "titles and prices" in query and "in stock" in query:
        return '''for $b in /books/book
where $b/availability = 'In stock'
return <book>
  <title>{ $b/title/text() }</title>
  <price>{ $b/price/text() }</price>
</book>'''

    match = re.search(r"books with a rating of '(\w+)'", query)
    if match:
        rating = match.group(1).capitalize()
        return f"for $b in /books/book\nwhere $b/rating = '{rating}'\nreturn $b/title"

    match = re.search(r"books in the '([\w\s]+)' category", query)
    if match:
        category = match.group(1)
        return f"for $b in /books/book\nwhere $b/category = '{category}'\nreturn $b/title"

    match = re.search(r"books that cost more than £(\d+(?:\.\d+)?)", query)
    if match:
        price = match.group(1)
        return f'''for $b in /books/book
let $p := xs:decimal(substring($b/price, 2))
where $p > {price}
return $b/title'''

    if "image urls of books rated" in query:
        rating = query.split("rated")[-1].strip().capitalize()
        return f"for $b in /books/book\nwhere $b/rating = '{rating}'\nreturn $b/'image-url'"

    return "Sorry, I don't understand that query."

def run_xquery(xquery_code, xml_path):
    cmd = ['basex', '-i', xml_path, xquery_code]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return result.stdout if result.returncode == 0 else result.stderr


In [20]:
nl = input("Ask a question about the XML: ")
xquery_code = nl_to_xquery(nl)
print("Generated XQuery:\n", xquery_code)


Generated XQuery:
 for $b in /books/book
where $b/rating = 'Four'
return $b/title
